In [10]:
import pandas as pd
import sklearn
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from fairness_evals import positive_rates, true_postive_rates, true_negative_rate, false_postive_rates
from Dataverwerking.load_data import get_old_data, get_reconstructed_data
from Preprocessing_2018 import preprocessing_2018

# Data

In [14]:

toy_educ = ['HS-grad', 'Doctorate', 'Masters', 'Bachelors', 'Doctorate'] * 3 
random.shuffle(toy_educ)
toy_mar = ['widowed', 'divorced', 'married', 'single', 'married'] * 3
random.shuffle(toy_mar)
toy_sex = ['Male', 'Male', 'Male', 'Female', 'Female'] * 3
random.shuffle(toy_sex)
toy_inc = [1, 0, 1, 0, 1] * 3
random.shuffle(toy_inc)
toy_educ_num = [31,41,17,24, 36,23,41,57,25,72,33,54,56,27,18]
random.shuffle(toy_educ_num)
toy_data = pd.DataFrame({'education':toy_educ , 'marital.status':toy_mar, 'gender':toy_sex, 'income':toy_inc, 'age':toy_educ_num})
toy_data

,education,marital.status,gender,income,age
0,Doctorate,divorced,Male,0,36
1,Masters,single,Female,1,57
2,Doctorate,married,Male,0,41
3,HS-grad,married,Female,0,27
4,Doctorate,married,Male,1,33
5,Doctorate,widowed,Male,1,23
6,Doctorate,single,Female,1,24
7,HS-grad,widowed,Female,1,54
8,Bachelors,divorced,Male,1,56
9,Masters,married,Male,0,17


In [15]:
# reconst_data = pd.read_csv('Dataverwerking/Adult_Reconstruction/adult_reconstruction.csv')

In [16]:
data_old = get_old_data("data/adult.csv")

/Users/jip/Documents/Uni_master/Human-Centered ML/HCML_project/Dataverwerking/load_data.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = old_data.applymap(lambda x: '?' in str(x))


In [17]:
data_old.keys()

Index(['age', 'workclass', 'education', 'marital.status', 'occupation',
       'relationship', 'race', 'sex', 'hours.per.week', 'income'],
      dtype='object')

In [18]:
data_old

,age,workclass,education,marital.status,occupation,relationship,race,sex,hours.per.week,income
1,82.0,Private,HS-grad,Widowed,Exec-managerial,Not-in-family,White,Female,18.0,False
3,54.0,Private,7th-8th,Divorced,Machine-op-inspct,Unmarried,White,Female,40.0,False
4,41.0,Private,Some-college,Separated,Prof-specialty,Own-child,White,Female,40.0,False
5,34.0,Private,HS-grad,Divorced,Other-service,Unmarried,White,Female,45.0,True
6,38.0,Private,10th,Separated,Adm-clerical,Unmarried,White,Male,40.0,False
...,...,...,...,...,...,...,...,...,...,...
30111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
30121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
30141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
30149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [19]:
data_2018 = preprocessing_2018()

In [20]:
data_2018.keys()

Index(['age', 'workclass', 'education', 'marital.status', 'occupation',
       'relationship', 'hours.per.week', 'sex', 'race', 'income'],
      dtype='object')

In [21]:
data_2018

,age,workclass,education,marital.status,occupation,relationship,hours.per.week,sex,race,income
1,18,Private,Some-college,Never-married,4720,17,21,Female,Black,False
2,53,Federal-gov,HS-grad,Never-married,3605,16,40,Male,White,False
8,41,Private,HS-grad,Never-married,7330,17,40,Male,White,False
9,18,Self-employed in own not incorporated business...,Some-college,Never-married,2722,17,2,Female,White,False
14,21,Federal-gov,Some-college,Never-married,3870,17,50,Male,White,False
...,...,...,...,...,...,...,...,...,...,...
38775,20,Self-employed in own not incorporated business...,Some-college,Never-married,4251,4,25,Male,White,False
38779,63,Private,HS-grad,Married,440,0,48,Male,White,True
38782,65,Private,Bachelors,Never-married,420,2,40,Female,White,True
38783,37,Private,HS-grad,Separated,340,0,50,Female,White,False


In [32]:
DATA = toy_data 
DATA_NAME = 'toy' #! don't forget to change this accordingly 
FEAT_OF_INT = ['education', 'marital-status', 'gender', 'income'] #The features we are interested in
DATA.head()

,education,marital.status,gender,income,age
0,Doctorate,divorced,Male,0,36
1,Masters,single,Female,1,57
2,Doctorate,married,Male,0,41
3,HS-grad,married,Female,0,27
4,Doctorate,married,Male,1,33


## Encoder for categorical values

In [23]:
from sklearn.preprocessing import OneHotEncoder
cat_feats = list(DATA.select_dtypes("object").keys())  #names of all categorical features
# cat_feats.remove('income') #keep these values as is 
print(f"{len(cat_feats)} categorical features found: {cat_feats}")

cat_feat_encoder = OneHotEncoder(sparse_output=False, drop='if_binary').set_output(transform="pandas")
cat_feat_encoder = cat_feat_encoder.fit(DATA[cat_feats])

3 categorical features found: ['education', 'marital.status', 'gender']


## Split train and test

In [24]:
train, test = train_test_split(DATA, test_size=0.3)
print(f"Train size: {len(train)}, test size: {len(test)}")

Train size: 10, test size: 5


In [25]:
x_train = train.drop(columns=['income'])
y_train = train['income']

x_test  = test.drop(columns=['income'])
y_test  = test['income']

In [26]:
from Dataverwerking.data_transform import cat_to_one_hot
x_train_encoded = cat_to_one_hot(x_train, cat_feats, cat_feat_encoder)
x_test_encoded = cat_to_one_hot(x_test, cat_feats, cat_feat_encoder)

# Logistic regression

**params logreg**
* Penalty (=L2)
* C (=1): regularization strenght -> set with dev set 
* solver: ?

In [27]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver = 'sag').fit(x_train_encoded, y_train)

y_pred= list(logreg.predict(x_test_encoded))

logreg_results = test.copy() #get all the data of the test set

logreg_results = logreg_results.filter(items=FEAT_OF_INT) #keep only relevant info
logreg_results['y_pred'] = y_pred
logreg_results

/Users/jip/Documents/Uni_master/Human-Centered ML/HCML_project/.venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


,education,gender,income,y_pred
5,Doctorate,Male,1,1
8,Bachelors,Male,1,1
12,Doctorate,Female,1,1
9,Masters,Male,0,1
1,Masters,Female,1,1


In [30]:
str(DATA)

'    education marital.status  gender  income  age\n0   Doctorate       divorced    Male       0   36\n1     Masters         single  Female       1   57\n2   Doctorate        married    Male       0   41\n3     HS-grad        married  Female       0   27\n4   Doctorate        married    Male       1   33\n5   Doctorate        widowed    Male       1   23\n6   Doctorate         single  Female       1   24\n7     HS-grad        widowed  Female       1   54\n8   Bachelors       divorced    Male       1   56\n9     Masters        married    Male       0   17\n10  Bachelors       divorced    Male       0   18\n11    HS-grad        widowed    Male       1   72\n12  Doctorate        married  Female       1   31\n13    Masters        married    Male       1   41\n14  Bachelors         single  Female       0   25'

In [34]:
logreg_acc = accuracy_score(y_test, y_pred)
logreg_rec = recall_score(y_test, y_pred)
logreg_pre = precision_score(y_test, y_pred)
logreg_f1 = f1_score(y_test, y_pred)
logreg_eval = pd.DataFrame({'accuracy': [logreg_acc], 'recall': [logreg_rec], 'precision': [logreg_pre], 'f1': [logreg_f1]}, index=['LogReg'])
logreg_eval.to_csv(f'Results/{DATA_NAME}_eval.csv',sep='\t')
logreg_eval

,accuracy,recall,precision,f1
LogReg,0.8,1.0,0.8,0.888889


In [36]:
logreg_fair_eval = pd.DataFrame()
logreg_fair_eval['positive rates'] = positive_rates(logreg_results, 'gender')
logreg_fair_eval['true positive rates'] = true_postive_rates(logreg_results, 'gender')
logreg_fair_eval['true negative rates'] = true_negative_rate(logreg_results, 'gender')
logreg_fair_eval['false positive rates'] = false_postive_rates(logreg_results, 'gender')
logreg_fair_eval.to_csv(f'Results/{DATA_NAME}_fair_eval.csv',sep='\t')
logreg_fair_eval


No gold false instances found for Female
No gold false instances found for Female


,positive rates,true positive rates,true negative rates,false positive rates
Male,1.0,1.0,0.0,1.0
Female,1.0,1.0,NaN,NaN


# Reweighing